In [1]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time

In [2]:
import numpy as np
import os

# Word processing libraries
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

# Widen the size of each cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
!pip install -U wordcloud

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import nltk
nltk.download('vader_lexicon')

In [ ]:
!pip install -U gensim

In [ ]:
!pip install preprocessor

In [ ]:
!pip install tweet-preprocessor

In [ ]:
auth = tweepy.OAuthHandler("QgYF0b9opJd9ZdFIaL9FProYG", 
    "KBU5QXBtO6SnL4CaQHuccvZYAFIRwjO2u2oE7wWmTcRnGf54vl")
auth.set_access_token("1030827643023675392-JNMBYqDZoXUACGvOu0KX4oVnhhNWTk", 
    "yJI9rO3OtsDwehDyljzEuvFJPIaizLHFssk8YNblVTKKM")

api = tweepy.API(auth)

function to fetch tweet and related required data and save it into csv

In [ ]:
def scraptweets(search_words, date_since, numTweets, numRuns):

    ## Arguments:
    # search_words -> define a string of keywords for this function to extract
    # date_since -> define a date from which to start extracting the tweets 
    # numTweets -> number of tweets to extract per run
    # numRun -> number of runs to perform in this program - API calls are limited to once every 15 mins, so each run will be 15 mins apart.
    ##
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    # Define a for-loop to generate tweets at regular intervals
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)

        # Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]

        # Obtain the following info (methods to call them out):
            # user.screen_name - twitter handle
            # user.description - description of account
            # user.location - where is he tweeting from
            # user.friends_count - no. of other users that user is following (following)
            # user.followers_count - no. of other users who are following this user (followers)
            # user.statuses_count - total tweets by user
            # user.created_at - when the user account was created
            # created_at - when the tweet was created
            # retweet_count - no. of retweets
            # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
            # retweeted_status.full_text - full text of the tweet
            # tweet.entities['hashtags'] - hashtags in the tweet

        # Begin scraping the tweets individually:
        noTweets = 0

        for tweet in tweet_list:

            # Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']

            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text

            # Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

            # Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet

            # increase counter - noTweets  
            noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round(end_run-start_run, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i, noTweets))
        print('time take for {} run to complete is {}'.format(i, duration_run))
        
        time.sleep(900) #15 minute sleep time

        
    # Once all runs have completed, save them to a single csv file:    
    # Obtain timestamp in a readable format:
    from datetime import datetime
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')

    # Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + 'corona_tweets.csv'

    # Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    print('Scraping has completed!')

In [ ]:
output=scraptweets('corona OR covid OR chinese virus OR #coronavirus OR #covid-19','2020-1-30',2000,2)

In [ ]:
import pandas as pd



## Load all the csv and than merge it

In [ ]:
tweets_1st = pd.read_csv(os.getcwd() + '/data/' + '/20200620_075152corona_tweets.csv')
tweets_2nd = pd.read_csv(os.getcwd() + '/data/' + '/20200620_090628corona_tweets.csv')
tweets_3rd = pd.read_csv(os.getcwd() + '/data/' + '/20200621_083711corona_tweets.csv')
tweets_4th = pd.read_csv(os.getcwd() + '/data/' + '/20200621_093813corona_tweets.csv')
tweets_5th = pd.read_csv(os.getcwd() + '/data/' + '/20200621_101115corona_tweets.csv')
tweets_6th = pd.read_csv(os.getcwd() + '/data/' + '/20200622_072958corona_tweets.csv')
tweets_7th = pd.read_csv(os.getcwd() + '/data/' + '/20200622_082446corona_tweets.csv')
tweets_8th = pd.read_csv(os.getcwd() + '/data/' + '/20200622_09315corona_tweets.csv')
tweets_9th = pd.read_csv(os.getcwd() + '/data/' + '/20200623_071902corona_tweets.csv')


##Analysis

In [ ]:
print('Size of 1st set is:', tweets_1st.shape)
print('Size of 2nd set is:', tweets_2nd.shape)
print('Size of 3rd set is:', tweets_3rd.shape)
print('Size of 4th set is:', tweets_4th.shape)
print('Size of 5th set is:', tweets_5th.shape)
print('Size of 6th set is:', tweets_6th.shape)
print('Size of 7th set is:', tweets_7th.shape)
print('Size of 8th set is:', tweets_8th.shape)
print('Size of 9th set is:', tweets_9th.shape)

In [ ]:

# Summary statistics
print(tweets_1st.info())
print('\n')
print(tweets_2nd.info())
print('\n')
print(tweets_3rd.info())
print('\n')
print(tweets_4th.info())
print('\n')
print(tweets_5th.info())
print('\n')
print(tweets_6th.info())
print('\n')
print(tweets_7th.info())
print('\n')
print(tweets_8th.info())
print('\n')
print(tweets_9th.info())
print('\n')

In [ ]:

# Concat the two dataset together:
data = pd.concat([tweets_1st, tweets_2nd, tweets_3rd,tweets_4th,tweets_5th,tweets_6th,tweets_7th,tweets_8th,tweets_9th], axis = 0)

print('Size of concatenated dataset is:', data.shape)

# Reset_index
data.reset_index(inplace = True, drop = True)
data.head()
print(data.info())

In [ ]:

# Check out the number of unique users in the dataset
# It can be seen that some users are more proactive on the social media platform than others.
data['username'].value_counts()

In [ ]:
# Let's drop duplicated rows:
print('Initial size of dataset before dropping duplicated rows:', data.shape)
data.drop_duplicates(keep = False, inplace = True)

print('Current size of dataset after dropping duplicated rows, if any, is:', data.shape)
print(data.head())

In [ ]:
data.dropna(subset = ['text'], inplace = True)



In [ ]:
data.info()

In [ ]:
data.head()

## Making copy of data

In [ ]:
data_copy=data

## Removing  non english words 

In [ ]:

# The unicode accounts for chinese  hindi arabiccharacters and punctuations.
def strip_nonenglish_words(string):
    # list of english words
    en_list = re.findall(u'[^\u4E00-\u9FA5\u3000-\u303F\u0900-\u097F\u0600-\u06FF]', str(string))
    
    # Remove word from the list, if not english
    for c in string:
        if c not in en_list:
            string = string.replace(c, '')
    return string

In [ ]:
# Apply strip_nonenglish_words(...) on the column 'text'
data['text'] = data['text'].apply(lambda x: strip_nonenglish_words(x))
data.head()

In [ ]:
data.info()

#Collect @Users mentioned in each tweet


We want to obtain this useful information from each tweet because it could allow us to analyse what are the popular figures in the covid situation

In [ ]:

# Define function to sieve out @users in a tweet:
def mentioned_users(string):
    usernames = re.findall('@[^\s]+', string)
    return usernames


# Create a new column and apply the function on the column 'text'
data['mentioned_users'] = data['text'].apply(lambda x: mentioned_users(x))
data.head()

## extraction of hastags


In [ ]:
# Import ast to convert a string representation of list to list
# The column 'hashtags' is affected
import ast

# Define a function to convert a string rep. of list to list
## Function should also handle NaN values after conversion
def strlist_to_list(text):
    
    # Remove NaN
    if pd.isnull(text) == True: # if true
        text = ''
    else:
        text = ast.literal_eval(text)
    
    return text

In [ ]:
# Apply strlist_to_list(...) to the column 'hashtags'
# Note that doing so will return a list of dictionaries, where there will be one dictionary for each hashtag in a single tweet.
data['hashtags'] = data['hashtags'].apply(lambda x: strlist_to_list(x))
data.head()

In [ ]:
# Since each 'hashtag' contain a list of dictionaries, we need to loop through the list to extract the actual hashtags in the tweets.
# Define a function to perform this extraction:
def extract_hashtags(hashtag_list):
    # argument:
    # hashtag_list - a list of dictionary(ies), each containing a hashtag
    
    # Create a list to store the hashtags
    hashtags = []
    
    # Loop through the list:
    for i in range(0, len(hashtag_list)):
        # extract the hashtag value using the key - 'text'
        # For our purposes, we can ignore the indices, which tell us the position of the hashtags in the string of tweet
        # lowercase the text as well
        hashtags.append(hashtag_list[i]['text'].lower())
        
    return hashtags

In [ ]:
# Apply function on the column - data['hashtags']
data['hashtags'] = data['hashtags'].apply(lambda x: extract_hashtags(x))

# Check out the updated column 'hashtags'
print(data.head()['hashtags'])

In [ ]:
# Replace NaN (empty) values with n.a to indicate that the user did not state his location
# Define a function to handle this:
def remove_nan(text):
    if pd.isnull(text) == True: # entry is NaN
        text = 'n.a'
    else:
        # lowercase text for possible easy handling
        text = text.lower()
        
    return text

In [ ]:
# Apply function on column - data['location']
data['location'] = data['location'].apply(lambda x: remove_nan(x))

# Check out the updated columns
print(data.head()['location'])

In [ ]:
# Let's take a quick look at the value_counts()
data['location'].value_counts()

In [ ]:

# Apply the function already defined above: remove_nan(...)
# Apply function on column - data['location']
data['acctdesc'] = data['acctdesc'].apply(lambda x: remove_nan(x))

# Check out the updated columns
print(data.head()['acctdesc'])

## Main Text Cleaning

In [ ]:
#cleaning emojis and others 
# Define Emoji_patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [ ]:

# Define the function to implement POS tagging:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def clean_text(text):
    
    # Apply regex expressions first before converting string to list of tokens/words:
    # 1. remove @usernames
    text = re.sub('@[^\s]+', '', text)
    
    # 2. remove URLs
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    
    # 3. remove hashtags entirely i.e. #hashtags
    text = re.sub(r'#([^\s]+)', '', text)
    
    # 4. remove emojis
    text = emoji_pattern.sub(r'', text)
    
    # 5. Convert text to lowercase
    text = text.lower()
    
    # 6. tokenize text and remove punctuation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    
    # 7. remove numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    
    # 8. remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    
    # 9. remove empty tokens
    text = [t for t in text if len(t) > 0]
    
    # 10. pos tag text and lemmatize text
    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    
    # 11. remove words with only one letter
    text = [t for t in text if len(t) > 1]
    
    # join all
    text = " ".join(text)
    
    return(text)

In [ ]:

# Apply function on the column 'text':
data['cleaned_text'] = data['text'].apply(lambda x: clean_text(x))
data.head()

In [ ]:
data.info()

## saving cleaned dataset

In [ ]:

data.to_csv('covid2019_0120_0220_cleans_final.csv',index=False)
data.head()

In [ ]:
data.info()

In [ ]:
data.tail()

In [ ]:
# Drop the row when clean_text is none
data_strip_none = data[data.cleaned_text.notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))

In [ ]:
data_strip_none.info()

In [ ]:
data_strip_none.to_csv('cleaned_text.csv',index=False)

In [ ]:
data_strip_none.info()

In [ ]:
!pip install ibm-watson

In [ ]:
!pip install langdetect

In [3]:

from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

In [ ]:

# random select 1000 samples
random.seed(4509)

data_api, data_test = train_test_split(data_strip_none, train_size=18505, random_state=0)

# Reset index
data_api = data_api.set_index(pd.Index(range(0,len(data_api))))
data_test = data_test.set_index(pd.Index(range(0,len(data_test))))

# Save the dataset
data_api.to_csv('data_api.csv',index=False)
data_test.to_csv('data_test_0120_0220.csv',index=False)

In [ ]:
data_api.info()

## checking train and test info

In [ ]:
data_test.info()

## new checkpoint


IBM Tone Analyzer

In [65]:
data_api = pd.read_csv('data_api.csv')


In [ ]:
data_api.info()

In [ ]:
data_api.head()

In [66]:
data_api_9th_set = data_api.iloc[4000:5000,]


In [67]:
data_api_9th_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 4000 to 4999
Data columns (total 13 columns):
username           1000 non-null object
acctdesc           1000 non-null object
location           1000 non-null object
following          1000 non-null int64
followers          1000 non-null int64
totaltweets        1000 non-null int64
usercreatedts      1000 non-null object
tweetcreatedts     1000 non-null object
retweetcount       1000 non-null int64
text               1000 non-null object
hashtags           1000 non-null object
mentioned_users    1000 non-null object
cleaned_text       1000 non-null object
dtypes: int64(4), object(9)
memory usage: 101.6+ KB


## IBM Tone analyzer api details

In [68]:

authenticator = IAMAuthenticator('pu51crsTePmHNUxqLUaPiTxHOqRSSwrAErfjGw13pHFx')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com/instances/47aeb973-3885-4d70-b2f0-e4fd4dea3b7d')
tone_analyzer.set_disable_ssl_verification(False)

In [69]:
data_api_9th_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 4000 to 4999
Data columns (total 13 columns):
username           1000 non-null object
acctdesc           1000 non-null object
location           1000 non-null object
following          1000 non-null int64
followers          1000 non-null int64
totaltweets        1000 non-null int64
usercreatedts      1000 non-null object
tweetcreatedts     1000 non-null object
retweetcount       1000 non-null int64
text               1000 non-null object
hashtags           1000 non-null object
mentioned_users    1000 non-null object
cleaned_text       1000 non-null object
dtypes: int64(4), object(9)
memory usage: 101.6+ KB


In [70]:
data_api_9th_set.tail()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,mentioned_users,cleaned_text
4995,PKSHRIV20592309,n.a,n.a,3,10,376,2020-01-14 07:25:17,2020-06-22 02:13:18,7,Webinar on HR challenges &amp; opportunity Dur...,[],"['@easterncoal', '@coalindia.in']",webinar hr challenge amp opportunity post hr p...
4996,thereal_tengar,n.a,"salt lake city, ut",59,29,637,2011-02-14 04:11:32,2020-06-21 02:21:17,160,"As we see COVID-19 cases spike, I urge all Uta...",[],[],see case spike urge utahns use face covering w...
4997,al808_al,aloha au'la ko'u ohana❤ / live aloha /\nhope i...,pacific time zone,946,617,3724,2015-08-23 08:25:09,2020-06-21 02:37:57,489,Breaking News: Covid Virus extremely confused ...,['trumprallyfail'],[],break news covid virus extremely confused i'm ...
4998,rjones2818,a librarian socialist/anarchist.,central indiana,4681,2026,37087,2008-10-05 03:06:27,2020-06-21 02:18:05,1,@LumpyLouish Turns out LeftieLucie/Luma923 is ...,[],['@LumpyLouish'],turn antivax conspiracy peddler think fake cla...
4999,___eog___,𝑓𝑜𝑜𝑑 𝐸𝑛𝑔𝑖𝑛𝑒𝑒𝑟 ...,"agdal riyad, royaume du maroc",964,1215,14398,2019-01-29 22:11:52,2020-06-22 01:21:46,23361,Lack of taste is a symptom of covid-19 https:/...,[],[],lack taste symptom


In [71]:
print(data_api_9th_set['cleaned_text'])

4000    donate campaign one large artist donation sinc...
4001              protest start effect chinese virus case
4002    friend’s dad develop fever four day ago extrem...
4003              someone need make covid virus mouse ear
4004    thank switzerland especially zermatt show soli...
4005    washington post sudden remove daily china viru...
4006                 huhuhu really miss life corona virus
4007    it’s absolute death industry keep act side we’...
4008    care ramp president admit ordered testing slow...
4009           kerala test positive sunday case three day
4010    kung-flu chinese virus \ndont bring racist car...
4011         break declare onwards covid call wuhan virus
4012    expert want stop test number won’t get high th...
4013    truthfully get flukey lucky pandemic amp city ...
4014    brook shot man drug dealer convict numerous fe...
4015    governor probably promise hand they’ll nvr get...
4016                 huhuhu really miss life corona virus
4017          

In [72]:
tones = []
index = 0
for item in data_api_9th_set['cleaned_text']:
    try:
        index = index + 1
        text = item
        tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
        tones.append(tone_analysis)
    except ApiException as ex:
        tones.append(None)
        print ("Method failed with status code " + str(ex.code) + ": " + ex.message)




In [74]:
tones_copy = tones.copy()
tones_copy 

[{'document_tone': {'tones': [{'score': 0.583359,
     'tone_id': 'joy',
     'tone_name': 'Joy'}]}},
 {'document_tone': {'tones': [{'score': 0.972852,
     'tone_id': 'analytical',
     'tone_name': 'Analytical'}]}},
 {'document_tone': {'tones': [{'score': 0.632674,
     'tone_id': 'sadness',
     'tone_name': 'Sadness'}]}},
 {'document_tone': {'tones': [{'score': 0.91961,
     'tone_id': 'tentative',
     'tone_name': 'Tentative'}]}},
 {'document_tone': {'tones': [{'score': 0.881685,
     'tone_id': 'joy',
     'tone_name': 'Joy'}]}},
 {'document_tone': {'tones': [{'score': 0.630541,
     'tone_id': 'sadness',
     'tone_name': 'Sadness'},
    {'score': 0.893364, 'tone_id': 'tentative', 'tone_name': 'Tentative'}]},
  'sentences_tone': [{'sentence_id': 0,
    'text': 'washington post sudden remove daily china virus death tracker front page covid death decrease nationwide?',
    'tones': [{'score': 0.511428,
      'tone_id': 'sadness',
      'tone_name': 'Sadness'}]},
   {'sentence_id'

In [75]:
len(tones_copy)

1000

In [76]:
data_api_9th_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 4000 to 4999
Data columns (total 13 columns):
username           1000 non-null object
acctdesc           1000 non-null object
location           1000 non-null object
following          1000 non-null int64
followers          1000 non-null int64
totaltweets        1000 non-null int64
usercreatedts      1000 non-null object
tweetcreatedts     1000 non-null object
retweetcount       1000 non-null int64
text               1000 non-null object
hashtags           1000 non-null object
mentioned_users    1000 non-null object
cleaned_text       1000 non-null object
dtypes: int64(4), object(9)
memory usage: 101.6+ KB


In [ ]:
##data_api_9th_set.drop(data_api_9th_set.tail(683).index,inplace=True) 

In [ ]:
#data_api_9th_set.info()

data_api_5th_set_copy=data_api_5th_set.drop(data_api_5th_set.loc[965:-1].index, inplace=True)

In [ ]:
#data_api_9th_set.tail() 

In [ ]:
#tones_copy.index(None)

an error occurred at index 1194 which returned none value so we have to delete index 1194 of our data_api_1st_set than we have to merge it properly

In [ ]:
#data_api_9th_set.head()

In [ ]:
#data_api_9th_set.reset_index()

In [78]:
data_api_9th_set_copy=data_api_9th_set

In [79]:
data_api_9th_set_copy.head()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,mentioned_users,cleaned_text
4000,somaily,"modelo, artista, estudiante y estudiante perio...",n.a,312,447,5579,2009-12-27 23:58:14,2020-06-23 01:28:26,551,.@BTS_twt and @BigHitEnt have donated $1M to t...,['crewnation'],"['@BTS_twt', '@BigHitEnt']",donate campaign one large artist donation sinc...
4001,JOHNSTE16060063,u. s. federal government sucks pol...,"michigan, usa",13151,12627,28965,2018-09-06 22:47:18,2020-06-20 03:07:41,0,@USATODAY So the protests are starting to have...,[],['@USATODAY'],protest start effect chinese virus case
4002,mijaziburger,cricket. elephants. history aur kuch mystery.,islamabad/london,145,652,35991,2011-10-07 14:35:19,2020-06-21 04:15:13,80,A friend’s dad developed a fever four days ago...,[],[],friend’s dad develop fever four day ago extrem...
4003,KnicksFTW_HOTK,"just here to talk knicks, giants and mets, but...","atlanta, ga",1110,771,10655,2019-02-02 14:03:48,2020-06-21 03:23:28,0,@shwinnypooh Someone needs to make a Covid Vir...,[],['@shwinnypooh'],someone need make covid virus mouse ear
4004,Charank21394924,n.a,n.a,3,0,95,2020-06-22 03:07:40,2020-06-22 03:38:07,6528,Thank you Switzerland 🇨🇭 especially Zermatt fo...,[],"['@zermatt_tourism', '@MySwitzerland_e']",thank switzerland especially zermatt show soli...


In [ ]:
#len(tones_copy)

In [80]:
data_api_9th_set_copy.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 4000 to 4999
Data columns (total 13 columns):
username           1000 non-null object
acctdesc           1000 non-null object
location           1000 non-null object
following          1000 non-null int64
followers          1000 non-null int64
totaltweets        1000 non-null int64
usercreatedts      1000 non-null object
tweetcreatedts     1000 non-null object
retweetcount       1000 non-null int64
text               1000 non-null object
hashtags           1000 non-null object
mentioned_users    1000 non-null object
cleaned_text       1000 non-null object
dtypes: int64(4), object(9)
memory usage: 101.6+ KB


In [81]:
data_api_9th_set_copy = data_api_9th_set.reset_index()

In [82]:
data_api_9th_set_copy.head()

,index,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,mentioned_users,cleaned_text
0,4000,somaily,"modelo, artista, estudiante y estudiante perio...",n.a,312,447,5579,2009-12-27 23:58:14,2020-06-23 01:28:26,551,.@BTS_twt and @BigHitEnt have donated $1M to t...,['crewnation'],"['@BTS_twt', '@BigHitEnt']",donate campaign one large artist donation sinc...
1,4001,JOHNSTE16060063,u. s. federal government sucks pol...,"michigan, usa",13151,12627,28965,2018-09-06 22:47:18,2020-06-20 03:07:41,0,@USATODAY So the protests are starting to have...,[],['@USATODAY'],protest start effect chinese virus case
2,4002,mijaziburger,cricket. elephants. history aur kuch mystery.,islamabad/london,145,652,35991,2011-10-07 14:35:19,2020-06-21 04:15:13,80,A friend’s dad developed a fever four days ago...,[],[],friend’s dad develop fever four day ago extrem...
3,4003,KnicksFTW_HOTK,"just here to talk knicks, giants and mets, but...","atlanta, ga",1110,771,10655,2019-02-02 14:03:48,2020-06-21 03:23:28,0,@shwinnypooh Someone needs to make a Covid Vir...,[],['@shwinnypooh'],someone need make covid virus mouse ear
4,4004,Charank21394924,n.a,n.a,3,0,95,2020-06-22 03:07:40,2020-06-22 03:38:07,6528,Thank you Switzerland 🇨🇭 especially Zermatt fo...,[],"['@zermatt_tourism', '@MySwitzerland_e']",thank switzerland especially zermatt show soli...



indices = []
for i in range(len(tones_copy)):

   if tones_copy[i] == None:
      indices.append(i)

print(indices)


In [84]:
res = list(filter(None, tones_copy)) 

In [85]:
rs=pd.DataFrame(res)

In [86]:
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
document_tone     1000 non-null object
sentences_tone    273 non-null object
dtypes: object(2)
memory usage: 15.7+ KB


In [87]:
rs.tail()

,document_tone,sentences_tone
995,"{'tones': [{'score': 0.624822, 'tone_id': 'joy...","[{'sentence_id': 0, 'text': 'webinar hr challe..."
996,"{'tones': [{'score': 0.576771, 'tone_id': 'sad...",NaN
997,"{'tones': [{'score': 0.654326, 'tone_id': 'sad...",NaN
998,"{'tones': [{'score': 0.536591, 'tone_id': 'sad...",NaN
999,"{'tones': [{'score': 0.955445, 'tone_id': 'ana...",NaN


In [40]:
#data_api_9th_set_copy =data_api_9th_set_copy.drop([165])


In [88]:
data_api_9th_set_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
index              1000 non-null int64
username           1000 non-null object
acctdesc           1000 non-null object
location           1000 non-null object
following          1000 non-null int64
followers          1000 non-null int64
totaltweets        1000 non-null int64
usercreatedts      1000 non-null object
tweetcreatedts     1000 non-null object
retweetcount       1000 non-null int64
text               1000 non-null object
hashtags           1000 non-null object
mentioned_users    1000 non-null object
cleaned_text       1000 non-null object
dtypes: int64(5), object(9)
memory usage: 109.5+ KB


In [89]:
data_api_9th_set_copy  = data_api_9th_set_copy.reset_index()

In [90]:
data_api_9th_set_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
level_0            1000 non-null int64
index              1000 non-null int64
username           1000 non-null object
acctdesc           1000 non-null object
location           1000 non-null object
following          1000 non-null int64
followers          1000 non-null int64
totaltweets        1000 non-null int64
usercreatedts      1000 non-null object
tweetcreatedts     1000 non-null object
retweetcount       1000 non-null int64
text               1000 non-null object
hashtags           1000 non-null object
mentioned_users    1000 non-null object
cleaned_text       1000 non-null object
dtypes: int64(6), object(9)
memory usage: 117.3+ KB


In [91]:
data_api_9th_set_final = pd.concat([data_api_9th_set_copy,rs],axis=1)
data_api_9th_set_final.to_csv('tones_folder/data_api_9th_set_final.csv',index=False,encoding='utf8')

In [92]:
data_api_9th_set_final.head()

,level_0,index,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,mentioned_users,cleaned_text,document_tone,sentences_tone
0,0,4000,somaily,"modelo, artista, estudiante y estudiante perio...",n.a,312,447,5579,2009-12-27 23:58:14,2020-06-23 01:28:26,551,.@BTS_twt and @BigHitEnt have donated $1M to t...,['crewnation'],"['@BTS_twt', '@BigHitEnt']",donate campaign one large artist donation sinc...,"{'tones': [{'score': 0.583359, 'tone_id': 'joy...",NaN
1,1,4001,JOHNSTE16060063,u. s. federal government sucks pol...,"michigan, usa",13151,12627,28965,2018-09-06 22:47:18,2020-06-20 03:07:41,0,@USATODAY So the protests are starting to have...,[],['@USATODAY'],protest start effect chinese virus case,"{'tones': [{'score': 0.972852, 'tone_id': 'ana...",NaN
2,2,4002,mijaziburger,cricket. elephants. history aur kuch mystery.,islamabad/london,145,652,35991,2011-10-07 14:35:19,2020-06-21 04:15:13,80,A friend’s dad developed a fever four days ago...,[],[],friend’s dad develop fever four day ago extrem...,"{'tones': [{'score': 0.632674, 'tone_id': 'sad...",NaN
3,3,4003,KnicksFTW_HOTK,"just here to talk knicks, giants and mets, but...","atlanta, ga",1110,771,10655,2019-02-02 14:03:48,2020-06-21 03:23:28,0,@shwinnypooh Someone needs to make a Covid Vir...,[],['@shwinnypooh'],someone need make covid virus mouse ear,"{'tones': [{'score': 0.91961, 'tone_id': 'tent...",NaN
4,4,4004,Charank21394924,n.a,n.a,3,0,95,2020-06-22 03:07:40,2020-06-22 03:38:07,6528,Thank you Switzerland 🇨🇭 especially Zermatt fo...,[],"['@zermatt_tourism', '@MySwitzerland_e']",thank switzerland especially zermatt show soli...,"{'tones': [{'score': 0.881685, 'tone_id': 'joy...",NaN


Formating the document tones

In [93]:
data = pd.read_csv('tones_folder/data_api_9th_set_final.csv')


In [94]:
data.head()

,level_0,index,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,mentioned_users,cleaned_text,document_tone,sentences_tone
0,0,4000,somaily,"modelo, artista, estudiante y estudiante perio...",n.a,312,447,5579,2009-12-27 23:58:14,2020-06-23 01:28:26,551,.@BTS_twt and @BigHitEnt have donated $1M to t...,['crewnation'],"['@BTS_twt', '@BigHitEnt']",donate campaign one large artist donation sinc...,"{'tones': [{'score': 0.583359, 'tone_id': 'joy...",NaN
1,1,4001,JOHNSTE16060063,u. s. federal government sucks pol...,"michigan, usa",13151,12627,28965,2018-09-06 22:47:18,2020-06-20 03:07:41,0,@USATODAY So the protests are starting to have...,[],['@USATODAY'],protest start effect chinese virus case,"{'tones': [{'score': 0.972852, 'tone_id': 'ana...",NaN
2,2,4002,mijaziburger,cricket. elephants. history aur kuch mystery.,islamabad/london,145,652,35991,2011-10-07 14:35:19,2020-06-21 04:15:13,80,A friend’s dad developed a fever four days ago...,[],[],friend’s dad develop fever four day ago extrem...,"{'tones': [{'score': 0.632674, 'tone_id': 'sad...",NaN
3,3,4003,KnicksFTW_HOTK,"just here to talk knicks, giants and mets, but...","atlanta, ga",1110,771,10655,2019-02-02 14:03:48,2020-06-21 03:23:28,0,@shwinnypooh Someone needs to make a Covid Vir...,[],['@shwinnypooh'],someone need make covid virus mouse ear,"{'tones': [{'score': 0.91961, 'tone_id': 'tent...",NaN
4,4,4004,Charank21394924,n.a,n.a,3,0,95,2020-06-22 03:07:40,2020-06-22 03:38:07,6528,Thank you Switzerland 🇨🇭 especially Zermatt fo...,[],"['@zermatt_tourism', '@MySwitzerland_e']",thank switzerland especially zermatt show soli...,"{'tones': [{'score': 0.881685, 'tone_id': 'joy...",NaN


In [95]:
data.head()

,level_0,index,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,mentioned_users,cleaned_text,document_tone,sentences_tone
0,0,4000,somaily,"modelo, artista, estudiante y estudiante perio...",n.a,312,447,5579,2009-12-27 23:58:14,2020-06-23 01:28:26,551,.@BTS_twt and @BigHitEnt have donated $1M to t...,['crewnation'],"['@BTS_twt', '@BigHitEnt']",donate campaign one large artist donation sinc...,"{'tones': [{'score': 0.583359, 'tone_id': 'joy...",NaN
1,1,4001,JOHNSTE16060063,u. s. federal government sucks pol...,"michigan, usa",13151,12627,28965,2018-09-06 22:47:18,2020-06-20 03:07:41,0,@USATODAY So the protests are starting to have...,[],['@USATODAY'],protest start effect chinese virus case,"{'tones': [{'score': 0.972852, 'tone_id': 'ana...",NaN
2,2,4002,mijaziburger,cricket. elephants. history aur kuch mystery.,islamabad/london,145,652,35991,2011-10-07 14:35:19,2020-06-21 04:15:13,80,A friend’s dad developed a fever four days ago...,[],[],friend’s dad develop fever four day ago extrem...,"{'tones': [{'score': 0.632674, 'tone_id': 'sad...",NaN
3,3,4003,KnicksFTW_HOTK,"just here to talk knicks, giants and mets, but...","atlanta, ga",1110,771,10655,2019-02-02 14:03:48,2020-06-21 03:23:28,0,@shwinnypooh Someone needs to make a Covid Vir...,[],['@shwinnypooh'],someone need make covid virus mouse ear,"{'tones': [{'score': 0.91961, 'tone_id': 'tent...",NaN
4,4,4004,Charank21394924,n.a,n.a,3,0,95,2020-06-22 03:07:40,2020-06-22 03:38:07,6528,Thank you Switzerland 🇨🇭 especially Zermatt fo...,[],"['@zermatt_tourism', '@MySwitzerland_e']",thank switzerland especially zermatt show soli...,"{'tones': [{'score': 0.881685, 'tone_id': 'joy...",NaN


In [96]:
data = data.iloc[:,1:19]


In [97]:
data['document_tones'] = data.document_tone.apply(lambda x: ast.literal_eval(x)['tones'])
data.drop('document_tone',axis = 1,inplace = True)

In [98]:
# split sentiments in corresponding columns
for i in range(0, len(data)):
    for j in range(0, len(data.loc[i, 'document_tones'])):
        dic = data.loc[i, 'document_tones'][j]
        source = dic['tone_id']
        data.loc[i, source] = 1
data.head()

,index,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,...,cleaned_text,sentences_tone,document_tones,joy,analytical,sadness,tentative,confident,fear,anger
0,4000,somaily,"modelo, artista, estudiante y estudiante perio...",n.a,312,447,5579,2009-12-27 23:58:14,2020-06-23 01:28:26,551,...,donate campaign one large artist donation sinc...,NaN,"[{'score': 0.583359, 'tone_id': 'joy', 'tone_n...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,4001,JOHNSTE16060063,u. s. federal government sucks pol...,"michigan, usa",13151,12627,28965,2018-09-06 22:47:18,2020-06-20 03:07:41,0,...,protest start effect chinese virus case,NaN,"[{'score': 0.972852, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,4002,mijaziburger,cricket. elephants. history aur kuch mystery.,islamabad/london,145,652,35991,2011-10-07 14:35:19,2020-06-21 04:15:13,80,...,friend’s dad develop fever four day ago extrem...,NaN,"[{'score': 0.632674, 'tone_id': 'sadness', 'to...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,4003,KnicksFTW_HOTK,"just here to talk knicks, giants and mets, but...","atlanta, ga",1110,771,10655,2019-02-02 14:03:48,2020-06-21 03:23:28,0,...,someone need make covid virus mouse ear,NaN,"[{'score': 0.91961, 'tone_id': 'tentative', 't...",NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,4004,Charank21394924,n.a,n.a,3,0,95,2020-06-22 03:07:40,2020-06-22 03:38:07,6528,...,thank switzerland especially zermatt show soli...,NaN,"[{'score': 0.881685, 'tone_id': 'joy', 'tone_n...",1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
data.drop(['index'], axis=1)

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,...,cleaned_text,sentences_tone,document_tones,joy,analytical,sadness,tentative,confident,fear,anger
0,somaily,"modelo, artista, estudiante y estudiante perio...",n.a,312,447,5579,2009-12-27 23:58:14,2020-06-23 01:28:26,551,.@BTS_twt and @BigHitEnt have donated $1M to t...,...,donate campaign one large artist donation sinc...,NaN,"[{'score': 0.583359, 'tone_id': 'joy', 'tone_n...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,JOHNSTE16060063,u. s. federal government sucks pol...,"michigan, usa",13151,12627,28965,2018-09-06 22:47:18,2020-06-20 03:07:41,0,@USATODAY So the protests are starting to have...,...,protest start effect chinese virus case,NaN,"[{'score': 0.972852, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,mijaziburger,cricket. elephants. history aur kuch mystery.,islamabad/london,145,652,35991,2011-10-07 14:35:19,2020-06-21 04:15:13,80,A friend’s dad developed a fever four days ago...,...,friend’s dad develop fever four day ago extrem...,NaN,"[{'score': 0.632674, 'tone_id': 'sadness', 'to...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,KnicksFTW_HOTK,"just here to talk knicks, giants and mets, but...","atlanta, ga",1110,771,10655,2019-02-02 14:03:48,2020-06-21 03:23:28,0,@shwinnypooh Someone needs to make a Covid Vir...,...,someone need make covid virus mouse ear,NaN,"[{'score': 0.91961, 'tone_id': 'tentative', 't...",NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,Charank21394924,n.a,n.a,3,0,95,2020-06-22 03:07:40,2020-06-22 03:38:07,6528,Thank you Switzerland 🇨🇭 especially Zermatt fo...,...,thank switzerland especially zermatt show soli...,NaN,"[{'score': 0.881685, 'tone_id': 'joy', 'tone_n...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,RareGemUSA,#sovereignty #nationalism #trumpsupporter #kag...,usa,4652,3484,63954,2014-03-31 01:32:45,2020-06-21 03:15:40,1688,Why did the Washington Post all of a sudden re...,...,washington post sudden remove daily china viru...,"[{'sentence_id': 0, 'text': 'washington post s...","[{'score': 0.630541, 'tone_id': 'sadness', 'to...",NaN,NaN,1.0,1.0,NaN,NaN,NaN
6,Krishnadums,"god gave me a lot of hair, but not a lot of he...","quezon, northern mindanao",263,278,6691,2017-08-21 10:51:42,2020-06-21 04:22:17,1377,huhuhu I really miss how life was before the c...,...,huhuhu really miss life corona virus,NaN,"[{'score': 0.88846, 'tone_id': 'sadness', 'ton...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
7,EJ_Brooklyn_Own,38 year old brooklyn guy with trini roots (🇹🇹)...,"brooklyn, ny",2003,1870,30698,2011-12-06 19:35:16,2020-06-23 01:09:08,2392,It’s absolute death for this industry to keep ...,...,it’s absolute death industry keep act side we’...,NaN,"[{'score': 0.667303, 'tone_id': 'sadness', 'to...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
8,SmithSeigel,always grateful to god in all ways.,n.a,659,209,3235,2020-02-12 02:38:38,2020-06-22 01:17:35,2686,I don't care at all about the ramp. The presi...,...,care ramp president admit ordered testing slow...,NaN,"[{'score': 0.727798, 'tone_id': 'confident', '...",NaN,NaN,NaN,NaN,1.0,NaN,NaN
9,TOICitiesNews,unmatched coverage from over 60 cities across ...,n.a,94,19633,729724,2009-04-13 11:14:45,2020-06-22 02:11:58,0,"Kerala: 133 test Covid-19 positive on Sunday, ...",...,kerala test positive sunday case three day,NaN,"[{'score': 0.765125, 'tone_id': 'joy', 'tone_n...",1.0,1.0,NaN,NaN,1.0,NaN,NaN


In [100]:
data.to_csv('sentimental_data/data_api_9th_set.csv',index=False, encoding='utf8')


## Merging all the sentimental dataset now to one dataset and cleaning useless  indexextra column before mreging

In [28]:
tweet_1st = pd.read_csv(os.getcwd() + '/sentimental_data/' + '/data_api_1st_set.csv',encoding='utf8')
tweet_2nd = pd.read_csv(os.getcwd() + '/sentimental_data/' + '/data_api_2nd_set.csv',encoding='utf8')
tweet_3rd = pd.read_csv(os.getcwd() + '/sentimental_data/' + '/data_api_3rd_set.csv',encoding='utf8')
tweet_4th = pd.read_csv(os.getcwd() + '/sentimental_data/' + '/data_api_4th_set.csv',encoding='utf8')
tweet_5th = pd.read_csv(os.getcwd() + '/sentimental_data/' + '/data_api_5th_set.csv',encoding='utf8')
tweet_6th = pd.read_csv(os.getcwd() + '/sentimental_data/' + '/data_api_6th_set.csv',encoding='utf8')
tweet_7th = pd.read_csv(os.getcwd() + '/sentimental_data/' + '/data_api_7th_set.csv',encoding='utf8')
tweet_8th = pd.read_csv(os.getcwd() + '/sentimental_data/' + '/data_api_8th_set.csv',encoding='utf8')


In [31]:
tweet_1st.head()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,...,cleaned_text,sentences_tone,document_tones,anger,analytical,sadness,confident,tentative,fear,joy
0,AMANOLESCO,second profile-former pinkerton guard -passion...,"dunvegan,ontario,canada",996,235,15654,2012-08-17 16:57:09,2020-06-21 04:08:30,10703,6 members of Trump's Tulsa advance staff have ...,...,member trump's tulsa advance staff test positi...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,edgarrallanpoe_,"oakland 🛫 san antonio , rarely use twitter but...","san antonio, tx",498,17,398,2016-02-28 09:04:39,2020-06-20 03:07:05,368,"Thanks a lot jackasses, due to the spike in Co...",...,thanks lot jackass due spike corona virus case...,NaN,"[{'score': 0.597553, 'tone_id': 'anger', 'tone...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,KevinRobbins9,"retired baker. current gardener, food preserve...","fort edward, ny",755,463,11888,2012-11-06 16:23:51,2020-06-21 02:35:48,1,@RepStefanik @EliseStefanik @KevinRobbins9 \n...,...,elise gall bad mouth criticize great gov handl...,NaN,"[{'score': 0.532616, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,gina_roemer,i am antifa. #blm #lgbqtally #countryoverparty...,"quintana roo, méxico",725,405,66519,2015-07-18 22:59:29,2020-06-20 02:01:35,80,Chris Hayes on U.S. COVID-19 response: \n\n“Th...,...,chris hayes u.s response \n\n“the last refuge ...,"[{'sentence_id': 0, 'text': 'chris hayes u.s r...","[{'score': 0.553139, 'tone_id': 'sadness', 'to...",NaN,1.0,1.0,1.0,NaN,NaN,NaN
4,CindidevichG,n.a,n.a,79,200,23929,2014-02-08 04:14:52,2020-06-21 02:21:50,802,Why did the Washington Post all of a sudden re...,...,washington post sudden remove daily china viru...,"[{'sentence_id': 0, 'text': 'washington post s...","[{'score': 0.630541, 'tone_id': 'sadness', 'to...",NaN,NaN,1.0,NaN,1.0,NaN,NaN


In [35]:
tweet_2nd=tweet_2nd.drop(['index'], axis=1)

In [37]:
tweet_3rd.head()

,level_0,index,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,...,cleaned_text,sentences_tone,document_tones,analytical,confident,anger,tentative,joy,sadness,fear
0,0,1001,INQUIRERSports,the official twitter account of inquirer sport...,manila,354,48838,164845,2009-11-11 08:20:00,2020-06-22 01:19:32,...,grigor dimitrov test positive lead cancellatio...,NaN,"[{'score': 0.532616, 'tone_id': 'analytical', ...",1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,1,1002,ottersponge,20 // they/them,n.a,238,18,6184,2015-07-21 15:12:35,2020-06-20 03:19:02,...,thanks lot jackass due spike corona virus case...,NaN,"[{'score': 0.597553, 'tone_id': 'anger', 'tone...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,2,1003,Doehawk1993,"mom, teacher, musician, listener, reader- wear...","traverse city, mi",378,27,11982,2017-01-20 03:59:02,2020-06-22 01:38:55,...,texas gov greg abbott announces he’ll hold bri...,NaN,"[{'score': 0.793846, 'tone_id': 'analytical', ...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1004,AliceVincit,"wine, politics, religion, art - all wrapped up...",eastern washington state,2118,830,30782,2012-05-03 23:00:21,2020-06-23 01:08:51,...,amid surge florida case fla gov desantis chang...,NaN,"[{'score': 0.663532, 'tone_id': 'confident', '...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,4,1005,gman_h,"anthropology lecturer, researcher in migration...","brisbane, queensland",1755,1664,5286,2011-03-23 03:10:35,2020-06-22 03:21:07,...,“medical citizenship” pandemic generate new fo...,NaN,"[{'score': 0.936902, 'tone_id': 'analytical', ...",1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
tweet_3rd=tweet_3rd.drop(['index','level_0'], axis=1)

In [41]:
tweet_3rd.head()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,...,cleaned_text,sentences_tone,document_tones,analytical,confident,anger,tentative,joy,sadness,fear
0,INQUIRERSports,the official twitter account of inquirer sport...,manila,354,48838,164845,2009-11-11 08:20:00,2020-06-22 01:19:32,1,Grigor Dimitrov has tested positive for COVID-...,...,grigor dimitrov test positive lead cancellatio...,NaN,"[{'score': 0.532616, 'tone_id': 'analytical', ...",1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,ottersponge,20 // they/them,n.a,238,18,6184,2015-07-21 15:12:35,2020-06-20 03:19:02,364,"Thanks a lot jackasses, due to the spike in Co...",...,thanks lot jackass due spike corona virus case...,NaN,"[{'score': 0.597553, 'tone_id': 'anger', 'tone...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,Doehawk1993,"mom, teacher, musician, listener, reader- wear...","traverse city, mi",378,27,11982,2017-01-20 03:59:02,2020-06-22 01:38:55,938,Texas Gov. Greg Abbott announces he’ll hold br...,...,texas gov greg abbott announces he’ll hold bri...,NaN,"[{'score': 0.793846, 'tone_id': 'analytical', ...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AliceVincit,"wine, politics, religion, art - all wrapped up...",eastern washington state,2118,830,30782,2012-05-03 23:00:21,2020-06-23 01:08:51,12535,"Amid surge in Florida COVID-19 cases, Fla. Gov...",...,amid surge florida case fla gov desantis chang...,NaN,"[{'score': 0.663532, 'tone_id': 'confident', '...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,gman_h,"anthropology lecturer, researcher in migration...","brisbane, queensland",1755,1664,5286,2011-03-23 03:10:35,2020-06-22 03:21:07,0,COVID-19 and “medical citizenship”: How the pa...,...,“medical citizenship” pandemic generate new fo...,NaN,"[{'score': 0.936902, 'tone_id': 'analytical', ...",1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
tweet_4th=tweet_4th.drop(['index'], axis=1)

In [44]:
tweet_4th.head()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,...,cleaned_text,sentences_tone,document_tones,sadness,tentative,analytical,confident,fear,joy,anger
0,tatiRobotine,n.a,"oceanside, ca",96,463,62087,2017-01-15 21:52:07,2020-06-21 04:03:27,1962,Why did the Washington Post all of a sudden re...,...,washington post sudden remove daily china viru...,"[{'sentence_id': 0, 'text': 'washington post s...","[{'score': 0.630541, 'tone_id': 'sadness', 'to...",1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,HannibalBarca65,big brains like small government #2a #schoolch...,n.a,5578,5070,202322,2016-01-27 02:53:22,2020-06-21 02:20:16,810,Why did the Washington Post all of a sudden re...,...,washington post sudden remove daily china viru...,"[{'sentence_id': 0, 'text': 'washington post s...","[{'score': 0.630541, 'tone_id': 'sadness', 'to...",1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,AlexJMontanez,tx 🐂 | dgo 🦂,n.a,269,627,42800,2011-06-20 09:11:56,2020-06-20 03:17:29,25955,"Trump's rally in Tulsa is indoors, with no soc...",...,trump's rally tulsa indoors social distance ma...,NaN,"[{'score': 0.642915, 'tone_id': 'analytical', ...",NaN,NaN,1.0,1.0,NaN,NaN,NaN
3,ashleymlester16,bruins revenge tour 2020,massachusets,1330,565,16741,2018-12-05 19:30:49,2020-06-20 03:16:16,1,Okay listen all I’m saying is Logan Mitchell w...,...,okay listen i’m say logan mitchell would cure ...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Trump_Nov3,never underestimate the passion & the love ame...,n.a,1974,1375,12523,2020-03-24 12:14:48,2020-06-21 03:47:39,0,@democracy_am @realDonaldTrump I hardly call 1...,...,hardly call people one end corona virus people...,NaN,"[{'score': 0.5538, 'tone_id': 'tentative', 'to...",NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [48]:
tweet_7th=tweet_7th.drop(['index'], axis=1)

In [50]:
tweet_7th.head()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,...,cleaned_text,sentences_tone,document_tones,sadness,joy,tentative,confident,analytical,fear,anger
0,yspyg78,"🌊🌊🌊retired nurse, author, activist, photograph...",n.a,7643,6964,37187,2015-10-24 00:25:03,2020-06-22 01:17:24,716,Florida is a COVID-19 apocalypse compliments o...,...,florida apocalypse compliment science-denier i...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tellez_rebecca,"momma protecting her children, i recommend eve...","california, usa",658,154,2856,2015-10-14 18:47:36,2020-06-23 01:08:41,0,@latimes Check out Erin nurse from epicenter h...,...,check erin nurse epicenter hospital nyc test n...,NaN,"[{'score': 0.759976, 'tone_id': 'sadness', 'to...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,masoodkayani1,n.a,n.a,319,49,543,2012-02-28 00:07:35,2020-06-21 02:45:30,375,"Feb2020, WHO announced an official name COVID-...",...,announce official name novel coronavirus outbr...,"[{'sentence_id': 0, 'text': 'announce official...","[{'score': 0.519278, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,LoriHPhoton,n.a,98008,239,92,12712,2013-04-30 21:53:40,2020-06-22 03:39:12,132,Former Food and Drug Administration Commission...,...,former food drug administration commissioner s...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Polly01639091,n.a,n.a,5000,1804,49251,2020-04-11 01:36:32,2020-06-21 02:25:15,763,Why did the Washington Post all of a sudden re...,...,washington post sudden remove daily china viru...,"[{'sentence_id': 0, 'text': 'washington post s...","[{'score': 0.630541, 'tone_id': 'sadness', 'to...",1.0,NaN,1.0,NaN,NaN,NaN,NaN


In [52]:
tweet_8th=tweet_8th.drop(['index'], axis=1)

In [53]:
tweet_8th.head()

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,...,cleaned_text,sentences_tone,document_tones,sadness,analytical,confident,joy,tentative,anger,fear
0,nmfogg,"midwestern born and raised, patient care coord...",united states,151,71,4032,2011-08-12 03:58:23,2020-06-21 03:43:41,38,@bgluckman People know that Covid kills. Ignor...,...,people know covid kill ignore common sense adv...,NaN,"[{'score': 0.649822, 'tone_id': 'sadness', 'to...",1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,k2boorse,n.a,"scottsdale, az",51,44,13660,2014-04-29 22:11:29,2020-06-23 01:26:33,3956,This is an incredibly misleading headline.\n\n...,...,incredibly mislead incubation period days.\n\n...,"[{'sentence_id': 0, 'text': 'incredibly mislea...","[{'score': 0.556616, 'tone_id': 'sadness', 'to...",1.0,1.0,1.0,NaN,NaN,NaN,NaN
2,ThirdTimesaCha1,"@jpapantonakis new account, after the last one...",n.a,521,179,1479,2020-01-07 22:40:03,2020-06-22 02:32:33,15,Did you know that of the nearly 1200 health pr...,...,know nearly health professional cuban involve ...,"[{'sentence_id': 0, 'text': 'know nearly healt...","[{'score': 0.532277, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,1.0,1.0,NaN,NaN
3,seewhyarizona,"a native of tempe, arizona, now residing in an...","chandler, arizona",3243,472,45423,2008-12-13 02:42:51,2020-06-20 02:02:23,370,More than one-third of the total confirmed #CO...,...,one-third total confirm case maricopa county c...,NaN,"[{'score': 0.634825, 'tone_id': 'confident', '...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,BlackfullaRoar,"we are the ultimate humanitarians, conservatio...",point lookout,625,291,2603,2010-01-12 04:52:01,2020-06-22 03:22:16,1,@Time4U2Know @Tori_R10 @gapatriot3 @VincentCry...,...,topic post anyway important know measles virus...,"[{'sentence_id': 0, 'text': 'topic post anyway...","[{'score': 0.888555, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [55]:

# Summary statistics
print(tweet_1st.info())
print('\n')
print(tweet_2nd.info())
print('\n')
print(tweet_3rd.info())
print('\n')
print(tweet_4th.info())
print('\n')
print(tweet_5th.info())
print('\n')
print(tweet_6th.info())
print('\n')
print(tweet_7th.info())
print('\n')
print(tweet_8th.info())
print('\n')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 22 columns):
username           2499 non-null object
acctdesc           2499 non-null object
location           2499 non-null object
following          2499 non-null int64
followers          2499 non-null int64
totaltweets        2499 non-null int64
usercreatedts      2499 non-null object
tweetcreatedts     2499 non-null object
retweetcount       2499 non-null int64
text               2499 non-null object
hashtags           2499 non-null object
mentioned_users    2499 non-null object
cleaned_text       2499 non-null object
sentences_tone     779 non-null object
document_tones     2499 non-null object
anger              91 non-null float64
analytical         943 non-null float64
sadness            731 non-null float64
confident          563 non-null float64
tentative          623 non-null float64
fear               103 non-null float64
joy                76 non-null float64
dtypes: float64(7), 

In [59]:
# Concat the two dataset together:
data = pd.concat([tweet_1st, tweet_2nd, tweet_3rd,tweet_4th,tweet_5th,tweet_6th,tweet_7th,tweet_8th],sort=True, axis = 0)

print('Size of concatenated dataset is:', data.shape)

# Reset_index
data.reset_index(inplace = True, drop = True)
data.head()
print(data.info())

Size of concatenated dataset is: (9458, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9458 entries, 0 to 9457
Data columns (total 22 columns):
acctdesc           9458 non-null object
analytical         4057 non-null float64
anger              272 non-null float64
cleaned_text       9458 non-null object
confident          1818 non-null float64
document_tones     9458 non-null object
fear               313 non-null float64
followers          9458 non-null int64
following          9458 non-null int64
hashtags           9458 non-null object
joy                601 non-null float64
location           9458 non-null object
mentioned_users    9458 non-null object
retweetcount       9458 non-null int64
sadness            2375 non-null float64
sentences_tone     2755 non-null object
tentative          2424 non-null float64
text               9458 non-null object
totaltweets        9458 non-null int64
tweetcreatedts     9458 non-null object
usercreatedts      9458 non-null object
username 

In [60]:
data.head()

,acctdesc,analytical,anger,cleaned_text,confident,document_tones,fear,followers,following,hashtags,...,mentioned_users,retweetcount,sadness,sentences_tone,tentative,text,totaltweets,tweetcreatedts,usercreatedts,username
0,second profile-former pinkerton guard -passion...,NaN,NaN,member trump's tulsa advance staff test positi...,NaN,[],NaN,235,996,[],...,[],10703,NaN,NaN,NaN,6 members of Trump's Tulsa advance staff have ...,15654,2020-06-21 04:08:30,2012-08-17 16:57:09,AMANOLESCO
1,"oakland 🛫 san antonio , rarely use twitter but...",NaN,1.0,thanks lot jackass due spike corona virus case...,NaN,"[{'score': 0.597553, 'tone_id': 'anger', 'tone...",NaN,17,498,[],...,[],368,NaN,NaN,NaN,"Thanks a lot jackasses, due to the spike in Co...",398,2020-06-20 03:07:05,2016-02-28 09:04:39,edgarrallanpoe_
2,"retired baker. current gardener, food preserve...",1.0,NaN,elise gall bad mouth criticize great gov handl...,NaN,"[{'score': 0.532616, 'tone_id': 'analytical', ...",NaN,463,755,[],...,"['@RepStefanik', '@EliseStefanik', '@KevinRobb...",1,NaN,NaN,NaN,@RepStefanik @EliseStefanik @KevinRobbins9 \n...,11888,2020-06-21 02:35:48,2012-11-06 16:23:51,KevinRobbins9
3,i am antifa. #blm #lgbqtally #countryoverparty...,1.0,NaN,chris hayes u.s response \n\n“the last refuge ...,1.0,"[{'score': 0.553139, 'tone_id': 'sadness', 'to...",NaN,405,725,[],...,[],80,1.0,"[{'sentence_id': 0, 'text': 'chris hayes u.s r...",NaN,Chris Hayes on U.S. COVID-19 response: \n\n“Th...,66519,2020-06-20 02:01:35,2015-07-18 22:59:29,gina_roemer
4,n.a,NaN,NaN,washington post sudden remove daily china viru...,NaN,"[{'score': 0.630541, 'tone_id': 'sadness', 'to...",NaN,200,79,[],...,[],802,1.0,"[{'sentence_id': 0, 'text': 'washington post s...",1.0,Why did the Washington Post all of a sudden re...,23929,2020-06-21 02:21:50,2014-02-08 04:14:52,CindidevichG


In [61]:

# Save the dataset
data.to_csv('merged_sentimental/sentiment12000full.csv',index=False)

In [62]:
# Let's drop duplicated rows:
print('Initial size of dataset before dropping duplicated rows:', data.shape)
data.drop_duplicates(keep = False, inplace = True)

print('Current size of dataset after dropping duplicated rows, if any, is:', data.shape)
print(data.head())

Initial size of dataset before dropping duplicated rows: (9458, 22)
Current size of dataset after dropping duplicated rows, if any, is: (8786, 22)
                                            acctdesc  analytical  anger  \
0  second profile-former pinkerton guard -passion...         NaN    NaN   
1  oakland 🛫 san antonio , rarely use twitter but...         NaN    1.0   
2  retired baker. current gardener, food preserve...         1.0    NaN   
3  i am antifa. #blm #lgbqtally #countryoverparty...         1.0    NaN   
4                                                n.a         NaN    NaN   

                                        cleaned_text  confident  \
0  member trump's tulsa advance staff test positi...        NaN   
1  thanks lot jackass due spike corona virus case...        NaN   
2  elise gall bad mouth criticize great gov handl...        NaN   
3  chris hayes u.s response \n\n“the last refuge ...        1.0   
4  washington post sudden remove daily china viru...        NaN   


In [63]:
data.to_csv('merged_sentimental/sentiment_final.csv',index=False)